In [6]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from src.logit_gb import LogitGB

In [5]:
with open('models/model_up.p', 'rb') as file:
    model_up = pickle.load(file)
with open('data/locations.p', 'rb') as file:
    locations = pickle.load(file)
with open('models/vec.p', 'rb') as file:
    vec = pickle.load(file)
with open('archive/predictor.p', 'rb') as file:
    predictor = pickle.load(file)
with open('archive/text.p', 'rb') as file:
    text = pickle.load(file)

In [3]:
with open('train/fires_train.p', 'rb') as file:
    target_train = pickle.load(file)

In [4]:
with open('train/target_matching_train.p', 'rb') as file:
    train = pickle.load(file)

In [5]:
predictor['has_existing_stories'] = (predictor['existing_stories'] > 0).astype(int)

In [6]:
with open('train/fires.p', 'rb') as file:
    fires = pickle.load(file)

In [98]:
predictor['fires_before'] = fires[:,0]
predictor['fires_after'] = fires[:,1]
predictor['fire_rate_before'] = 365*predictor['fires_before']/(predictor['status_date'] - pd.to_datetime('2003-01-01')).apply(lambda x: x.days)
predictor['fire_rate_after'] = 365*predictor['fires_after']/(pd.to_datetime('2019-02-28') - predictor['status_date']).apply(lambda x: x.days)

In [8]:
predictor['existing_stories'] = predictor['existing_stories'].fillna(0)

In [17]:
test_mask = (predictor['status_date'].apply(lambda x: x.year) > 2017).values

In [18]:
df_test = predictor.iloc[test_mask]
df_test['fire_rate_after'] = 365*df_test['fires_after']/(pd.to_datetime('2019-2-28') - df_test['status_date']).apply(lambda x: x.days)

In [109]:
X_text_test = vec.transform(text[test_mask])

In [17]:
predictor_train = predictor.iloc[train.index]

In [18]:
n = 30000
sample = np.random.choice(predictor_train.shape[0], n)

In [42]:
target_train.shape

(351427, 2)

In [29]:
text_df_train = text[train.index][sample]

In [30]:
text_mat = vec.transform(text_df_train)

NameError: name 'text_df_train' is not defined

In [79]:
numerical_df = predictor_train.iloc[sample].fillna(0)

model = LogitGB()

In [73]:
target_train = predictor_train[['status_date', 'fires_after', 'fire_rate_after', 'fires_before', 'fire_rate_before']]

In [68]:
with open('data/full_train_data.p', 'wb') as file:
    pickle.dump(predictor_train, file)

In [29]:
text_mat

NameError: name 'text_mat' is not defined

In [80]:
model.fit(text_mat, numerical_df, target_train.iloc[sample])

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [87]:
X_text = vec.transform(text)

In [26]:
y_test = (df_test['fire_rate_after'] > df_test['fire_rate_before']).astype(int)

In [39]:
roc_auc_score(y_test, model_up.predict_proba(df_test, X_text_test))

0.8625379891729631

In [41]:
def f1(y_test, model, threshold):
    return f1_score(y_test, model.predict_proba(df_test, X_text_test) > threshold)

In [55]:
f1(y_test, model_up, .90)

0.3725910064239829

In [69]:
train = (predictor['status_date'].apply(lambda x: x.year) < 2015).values

In [63]:
train_idx = np.random.choice(len(text), int(len(text)*.7), replace = False)

In [64]:
for idx in train_idx:
    train[idx] = True

In [77]:
test = (predictor['status_date'].apply(lambda x: x.year) < 2017).values

In [84]:
test = (test & ~train).sum()

In [105]:
sample_idx = np.random.choice(train.sum(), 20000)

In [102]:
model_up = LogitGB()

In [106]:
model_up.fit(X_text[train][sample_idx], predictor[train].iloc[sample_idx], predictor[train].iloc[sample_idx])

C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pengj\Anaconda2\envs\threesix\lib\site-packages\

In [116]:
model_up.predict(predictor[test], X_text[test])

array([0, 0, 1, ..., 0, 0, 0])

In [117]:
roc_auc_score()

0.287

In [126]:
y = (predictor['fire_rate_after'] > predictor['fire_rate_before']).astype(int).values

In [127]:
y[test]

array([0, 0, 0, ..., 0, 0, 0])

In [132]:
roc_auc_score(y[test], model_up.predict_proba(predictor[test], X_text[test]))

0.7577431743636428

In [131]:
with open('models/model_up_trained_before_2015.p', 'wb') as file:
    pickle.dump(model_up, file)